<a href="https://colab.research.google.com/github/hamish-haggerty/AI-hacking/blob/master/SSL/cancer_random_initial_weights.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# cancer_random_initial_weights

> Ok, purpose of this notebook is to do the following:

    1) Fine tune models across two conditions: pretrain=True and pretrain=False. We show that ensembling across conditions is better than within. Also, importantly, save all the results: Then we can re-use e.g. the BT trained nets (but not fine tuned) in other ensemble experiments. Cool.
    2) Check list: copy paste `main_train` in temporarily, as well as `predict` (average in probability space rather than score space for stability). ALSO importantly try and ensure reproducibility by setting the appropriate flags (seed_everything etc)


Speaking a bit more generally, the plan of attack for the rest of today is the following: Work towards training final models for this notebook and the supervised vs self-supervised intitial weights. Tinker with the decorrelation idea concurrently, but we are not going to spend too long on that. It either works or it doesn't, basically. Then perhaps by tomorrow we will be training our final models on the decorrelation idea as well. So, by THURSDAY we can have 100% (basically) of our results, and writing the paper can continue in earnest. Getting the results is the hard part, once we have that we can do shit loads of writing in 1 week alone (but we don't want to leave it too late). 

You can do it, but you have to work well from this point on. The key is consistency, not intensity. So what, you procrastinated for an hour. So long as you then work well for a few hours. Cool.

In [1]:
#| default_exp cancer_random_initial_weights

In [2]:
#| hide
import os
from google.colab import drive

def colab_is_true():

    try: 
        from google.colab import drive

        return True 
    except ModuleNotFoundError:
        return False

def setup_colab():

    drive.mount('/content/drive',force_remount=True)
    #os.system('unzip -q "/content/drive/My Drive/archive (1).zip"')
    os.system('git clone https://github.com/hamish-haggerty/cancer-proj.git')

    os.chdir('cancer-proj')
    
    os.system('pip install .')
    os.system('pip install -qU nbdev')
    os.system('nbdev_install_quarto')

    os.system('unzip -q "/content/drive/My Drive/archive (1).zip"')

if __name__ == "__main__":
    on_colab = colab_is_true()
    if on_colab:
        setup_colab()

Mounted at /content/drive


In [3]:
#| hide
from nbdev.showdoc import *

In [4]:
#| export
from fastai.vision.all import *
from base_rbt.all import *
from cancer_proj.cancer_dataloading import *
from cancer_proj.cancer_metrics import *
from cancer_proj.cancer_maintrain import *

## Load the data

In [5]:
#| hide

#Since we have cloned repository and cd'd into it (and the data itself is not stored in the
#repo) we need cd out of it, get the data, then cd back into the repo `cancer-proj`.
#This is a bit annoying, can maybe remove this later
if on_colab:
    #os.chdir('..') #assumes we are currently in cancer-proj directory
    train_dir = colab_train_dir
    test_dir = colab_test_dir
else:
    train_dir = local_train_dir
    test_dir = local_test_dir

#define general hps
device ='cuda' if torch.cuda.is_available() else 'cpu'
#bs=256
#bs=698
bs=256
bs_tune=256
size=128
bs_val=174

#get the data dictionary
data_dict = get_fnames_dls_dict(train_dir=train_dir,test_dir=test_dir,
                    device=device,bs_val=bs_val,bs=bs,bs_tune=bs_tune,size=size,n_in=3)

#get the dataloaders
dls_train,dls_tune,dls_valid = data_dict['dls_train'],data_dict['dls_tune'],data_dict['dls_valid']
x,y = data_dict['x'],data_dict['y']
xval,yval = data_dict['xval'],data_dict['yval']
xtune,ytune = data_dict['xtune'],data_dict['ytune']
vocab = data_dict['vocab']

#If we want to write some tests (make sure the data is same every time etc):
fnames,fnames_train,fnames_tune,fnames_valid,fnames_test = data_dict['fnames'],data_dict['fnames_train'],data_dict['fnames_tune'],data_dict['fnames_valid'],data_dict['fnames_test']

test_eq(x.shape,xtune.shape)

# if on_colab:
#     os.chdir('cancer-proj')

## Load aug pipelines here

In [6]:
#| hide

aug_dict = create_aug_pipelines(size=size,device=device,Augs=BYOL_Augs,TUNE_Augs=TUNE_Augs,Val_Augs=Val_Augs)
aug_pipelines = aug_dict['aug_pipelines']
aug_pipelines_tune = aug_dict['aug_pipelines_tune']
aug_pipelines_test = aug_dict['aug_pipelines_test'] 

## Optionally, display:

In [7]:
#| hide
#show_bt_batch(dls=dls_train,aug=aug_pipelines,n_in=3)

In [8]:
#| hide

#show_linear_batch(dls=dls_tune,n_in=3,aug=aug_pipelines_tune,n=2,print_augs=True)

In [9]:
#| export

@patch
def lf(self:BarlowTwins, pred,*yb): return lf_bt(pred,I=self.I,lmb=self.lmb)

## Note: using this default encoder, for random weights, doesn't seem to work real well.

In [10]:
#| export

# """@patch
# @delegates(Learner.fit_one_cycle)
# def encoder_fine_tune(self:Learner, epochs, base_lr=2e-3, freeze_epochs=1, lr_mult=100,
#               pct_start=0.3, div=5.0, **kwargs):
#     "Fine tuner to use with bt initial weights"
    
#     #self.freeze() #freeze the resnet
#     self.fit_one_cycle(freeze_epochs, slice(base_lr), pct_start=0.99, **kwargs)
#     base_lr /= 2
#     #self.unfreeze() #unfreeze the resnet. 
#     #self.fit_one_cycle(epochs, slice(base_lr/lr_mult, base_lr), pct_start=pct_start, div=div, **kwargs)
#     self.fit_one_cycle(epochs, slice(base_lr, base_lr), pct_start=pct_start, div=div, **kwargs)

#     self.unfreeze() #We can unfreeze at the end"""

## Note: we don't actually want to freeze the resnet when pretraining in this notebook, since the initial weights are random. So we just need to make sure encoder_fine_tune isn't doing that

## Test: make sure that it freezes the appropriate part of model:

Make sure resnet frozen, linear head unfrozen:

In [11]:
bt_model,encoder = create_model(which_model='bt_pretrain',ps=8192,device=device)
model = LM(encoder)
test_eq(len(my_splitter(model)),2)
test_eq(len(my_splitter_bt(bt_model)),2)

learn = Learner(dls_tune,model,splitter=my_splitter,cbs = [LinearBt(aug_pipelines=aug_pipelines_tune,n_in=3)],wd=0.0)
learn.freeze()
print('body should be frozen, (sans batchnorm) linear head unfrozen')
learn.summary()

Downloading: "https://github.com/facebookresearch/barlowtwins/zipball/main" to /root/.cache/torch/hub/main.zip
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://dl.fbaipublicfiles.com/barlowtwins/ep1000_bs2048_lrw0.2_lrb0.0048_lambd0.0051/resnet50.pth" to /root/.cache/torch/hub/checkpoints/resnet50.pth


  0%|          | 0.00/90.0M [00:00<?, ?B/s]

body should be frozen, (sans batchnorm) linear head unfrozen


LM (Input shape: 256 x 3 x 128 x 128)
Layer (type)         Output Shape         Param #    Trainable 
                     256 x 64 x 64 x 64  
Conv2d                                    9408       False     
BatchNorm2d                               128        True      
ReLU                                                           
____________________________________________________________________________
                     256 x 64 x 32 x 32  
MaxPool2d                                                      
Conv2d                                    4096       False     
BatchNorm2d                               128        True      
Conv2d                                    36864      False     
BatchNorm2d                               128        True      
____________________________________________________________________________
                     256 x 256 x 32 x 32 
Conv2d                                    16384      False     
BatchNorm2d                               

## We can use this to check the  bt-model is of the appropriate type: resnet + projector

In [12]:
#| hide

#test : manual. BT

learn = Learner(dls_train,bt_model,splitter=my_splitter_bt,cbs=[BarlowTwins(aug_pipelines,n_in=3,lmb=1/8192,print_augs=False)])
learn.freeze()
print('body should be frozen, (sans batchnorm) projector unfrozen')
learn.summary()

body should be frozen, (sans batchnorm) projector unfrozen


BarlowTwinsModel (Input shape: 256 x 3 x 128 x 128)
Layer (type)         Output Shape         Param #    Trainable 
                     256 x 64 x 64 x 64  
Conv2d                                    9408       False     
BatchNorm2d                               128        True      
ReLU                                                           
____________________________________________________________________________
                     256 x 64 x 32 x 32  
MaxPool2d                                                      
Conv2d                                    4096       False     
BatchNorm2d                               128        True      
Conv2d                                    36864      False     
BatchNorm2d                               128        True      
____________________________________________________________________________
                     256 x 256 x 32 x 32 
Conv2d                                    16384      False     
BatchNorm2d                 

## Ok, so `fit_one_cycle` works the best! So let's patch it back in. Also, just need to add `pretrain` argument to `fine_tune`

We copy paste a few things in here for now, but need to commit to github once OS is updated:

In [13]:
#need to put this in a commit...

class main_train:
    """Instantiate and (optionally) train the encoder. Then fine-tune the supervised model. 
    Outputs metrics on validation data"""

    def __init__(self,
                 dls_train, #used for training BT (if pretrain=True)
                 dls_tune , #used for tuning
                 dls_valid, #used to compute metrics / evaluate results. 
                 xval, #currently `predict_model` below assumes this is entire validation / test data
                 yval,
                 aug_pipelines, #the aug pipeline for self-supervised learning
                 aug_pipelines_tune, #the aug pipeline for supervised learning
                 aug_pipelines_test, #test (or valid) time augmentations 
                 initial_weights, #Which initial weights to use
                 pretrain, #Whether to fit BT
                 num_epochs, #number of BT fit epochs
                 numfit, #number of tune_fit epochs
                 freeze_num_epochs, #How many epochs to freeze body for when training BT
                 freeze_numfit, #How many epochs to freeze body for when fine tuning
                 ps=8192, #projection size
                 n_in=3, #color channels
                 indim=2048, #dimension output of encoder (2048 for resnet50)
                 outdim=9, #number of classes
                 print_report=False, #F1 metrics etc
                 print_plot=False, #ROC curve
                 tune_model_path=None, #save models after fine tuning
                 ):
        store_attr()
        self.vocab = self.dls_valid.vocab
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

 
    @staticmethod
    def fit(learn,fit_type,epochs,freeze_epochs,initial_weights):
        """We can patch in a modification, e.g. if we want subtype of fine_tune:supervised_pretrain to be different
        to fine_tune:bt_pretrain"""

        if fit_type == 'encoder_fine_tune': #i.e. barlow twins

            learn.encoder_fine_tune(epochs,freeze_epochs=freeze_epochs) 

        elif fit_type == 'fine_tune':
            
            #elif initial_weights == 'supervised_pretrain':
            learn.linear_fine_tune(epochs,freeze_epochs=freeze_epochs) 
            #learn.fine_tune(epochs,freeze_epochs=freeze_epochs)

        else: raise Exception('Fit policy not of expected form')

    def train_encoder(self):
        "create encoder and (optionally, if pretrain=True) train with BT algorithm, according to fit_policy"

        try: #get existing encoder and plonk on new projector
            encoder = self.encoder
            encoder.cpu()
            bt_model = create_barlow_twins_model(encoder, hidden_size=self.ps,projection_size=self.ps,nlayers=3)
            bt_model.cuda()

        except AttributeError: #otherwise, create
            bt_model,encoder = create_model(which_model=self.initial_weights,ps=self.ps,device=self.device)

        if self.pretrain: #train encoder according to fit policy

            learn = Learner(self.dls_train,bt_model,splitter=my_splitter_bt,cbs=[BarlowTwins(self.aug_pipelines,n_in=self.n_in,lmb=1/self.ps,print_augs=False)])
            main_train.fit(learn,fit_type='encoder_fine_tune',
                           epochs=self.num_epochs,freeze_epochs=self.freeze_num_epochs,
                           initial_weights=self.initial_weights
                          )
            
        self.encoder = bt_model.encoder
        self.bt_model = bt_model

    def fine_tune(self):
        "fine tune in supervised fashion, according to tune_fit_policy, and get metrics"

        #encoder = pickle.loads(pickle.dumps(self.encoder)) #We might want to pretrain once and fine tune several times (varying e.g. tune augs)

        try: 
            encoder = self.encoder
        
        except AttributeError:
            _,self.encoder = create_model(which_model=self.initial_weights,ps=self.ps,device=self.device)

        model = LM(self.encoder)
        learn = Learner(self.dls_tune,model,splitter=my_splitter,cbs = [LinearBt(aug_pipelines=self.aug_pipelines_tune,n_in=self.n_in)],wd=0.0)
        
        #debugging
        #learn = Learner(self.dls_tune,model,cbs = [LinearBt(aug_pipelines=self.aug_pipelines_tune,n_in=self.n_in)],wd=0.0)

        main_train.fit(learn,fit_type='fine_tune',
                       epochs=self.numfit,freeze_epochs=self.freeze_numfit,
                       initial_weights=self.initial_weights
                      ) #fine tuning (don't confuse this with fit policy!)
        scores,preds, acc = predict_model(self.xval,self.yval,model=model,aug_pipelines_test=self.aug_pipelines_test,numavg=10)
        #metrics dict will have f1 score, auc etc etc
        metrics = classification_report_wrapper(preds, self.yval, self.vocab, print_report=self.print_report)
        auc_dict = plot_roc(self.yval,preds,self.vocab,print_plot=self.print_plot)
        metrics['acc'],metrics['auc_dict'],metrics['scores'],metrics['preds'],metrics['xval'],metrics['yval'] = acc,auc_dict,scores,preds,self.xval,self.yval
  

        if self.tune_model_path != None:
            metrics['classif_model_path'] = self.tune_model_path
            torch.save(model.state_dict(), self.tune_model_path)
        return metrics #

    def __call__(self):

        self.train_encoder() #train (or extract) the encoder
        metrics = self.fine_tune()
        
        return metrics


In [14]:
@patch_to(main_train)
def fit(learn,fit_type,epochs,freeze_epochs,initial_weights,pretrain):
    """We can patch in a modification, e.g. if we want subtype of fine_tune:supervised_pretrain to be different
    to fine_tune:bt_pretrain"""

    if fit_type == 'encoder_fine_tune': #i.e. barlow twins

        #learn.encoder_fine_tune(epochs,freeze_epochs=freeze_epochs)
        lr_max=0.0030199517495930195
        print(f'lr_max={lr_max}')
        learn.fit_one_cycle(epochs,lr_max= lr_max)

    elif fit_type == 'fine_tune':

        if pretrain == False:
            print('pretrain was False, and about to fit_one_cycle')
            learn.fit_one_cycle(epochs,lr_max=0.00027542) 

        elif pretrain == True:
            print('pretrain was True, and about to linear_fine_tune')
            learn.linear_fine_tune(epochs,freeze_epochs=freeze_epochs) #This gave very similar performance, when pretrain=False (see above / earlier commit)

        #learn.no_freeze_linear_fine_tune(epochs,freeze_epochs=freeze_epochs) 

    else: raise Exception('Fit policy not of expected form')


@patch
def fine_tune(self:main_train):
    "fine tune in supervised fashion, according to tune_fit_policy, and get metrics"

    #encoder = pickle.loads(pickle.dumps(self.encoder)) #We might want to pretrain once and fine tune several times (varying e.g. tune augs)

    try: 
        encoder = self.encoder
    
    except AttributeError:
        _,self.encoder = create_model(which_model=self.initial_weights,ps=self.ps,device=self.device)

    model = LM(self.encoder)
    learn = Learner(self.dls_tune,model,splitter=my_splitter,cbs = [LinearBt(aug_pipelines=self.aug_pipelines_tune,n_in=self.n_in)],wd=0.0)

    #debugging
    #learn = Learner(self.dls_tune,model,cbs = [LinearBt(aug_pipelines=self.aug_pipelines_tune,n_in=self.n_in)],wd=0.0)

    main_train.fit(learn,fit_type='fine_tune',
                    epochs=self.numfit,freeze_epochs=self.freeze_numfit,
                    initial_weights=self.initial_weights,
                    pretrain=self.pretrain
                    ) #fine tuning (don't confuse this with fit policy!)
    scores,preds, acc = predict_model(self.xval,self.yval,model=model,aug_pipelines_test=self.aug_pipelines_test,numavg=3)
    #metrics dict will have f1 score, auc etc etc
    metrics = classification_report_wrapper(preds, self.yval, self.vocab, print_report=self.print_report)
    auc_dict = plot_roc(self.yval,preds,self.vocab,print_plot=self.print_plot)
    metrics['acc'],metrics['auc_dict'],metrics['scores'],metrics['preds'],metrics['xval'],metrics['yval'] = acc,auc_dict,scores,preds,self.xval,self.yval

    if self.tune_model_path != None:
        metrics['classif_model_path'] = self.tune_model_path
        torch.save(model.state_dict(), self.tune_model_path)
     
    return metrics 

@patch
def train_encoder(self:main_train):
    "create encoder and (optionally, if pretrain=True) train with BT algorithm, according to fit_policy"

    try: #get existing encoder and plonk on new projector
        encoder = self.encoder
        encoder.cpu()
        bt_model = create_barlow_twins_model(encoder, hidden_size=self.ps,projection_size=self.ps,nlayers=3)
        bt_model.cuda()

    except AttributeError: #otherwise, create
        bt_model,encoder = create_model(which_model=self.initial_weights,ps=self.ps,device=self.device)

    if self.pretrain: #train encoder according to fit policy

        #lmb=0.005 #this guy doesn't seem to work
        lmb=1/8192
        print(f'lmb={lmb}')
        learn = Learner(self.dls_train,bt_model,cbs=[BarlowTwins(self.aug_pipelines,n_in=self.n_in,lmb=lmb,print_augs=False)])
        main_train.fit(learn,fit_type='encoder_fine_tune',
                        epochs=self.num_epochs,freeze_epochs=self.freeze_num_epochs,
                        initial_weights=self.initial_weights,
                        pretrain=self.pretrain
                        )
        
    self.encoder = bt_model.encoder
    self.bt_model=bt_model

## Reproducibility

In [15]:
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [16]:
#| export
import pickle

def save_dict_to_gdrive(d,directory, filename):
    #e.g. directory='/content/drive/My Drive/random_initial_weights'
    filepath = directory + '/' + filename + '.pkl'
    with open(filepath, "wb") as f:
        pickle.dump(d, f)

def load_dict_from_gdrive(directory,filename):
    #e.g. directory='/content/drive/My Drive/random_initial_weights'
    filepath = directory + '/' + filename + '.pkl'
    with open(filepath, "rb") as f:
        d = pickle.load(f)
    return d


In [17]:
import os

directory = '/content/drive/My Drive/random_initial_weights'

if not os.path.exists(directory):
    os.makedirs(directory)

In [18]:
def run_main_train(pretrain,initial_weights='no_pretrain',num_epochs=300,freeze_num_epochs=1,freeze_numfit=6,numfit=75,num=5):
    "run main_train num times."


    main_dict = {}
    for i in range(num):

        main = main_train(dls_train=dls_train,dls_tune=dls_tune,dls_valid=dls_valid, xval=xval, yval=yval,
                aug_pipelines=aug_pipelines, aug_pipelines_tune=aug_pipelines_tune, aug_pipelines_test=aug_pipelines_test, 
                initial_weights=initial_weights,pretrain=pretrain,
                num_epochs=num_epochs,numfit=numfit,freeze_num_epochs=freeze_num_epochs,freeze_numfit=freeze_numfit,
                print_report=True,
                tune_model_path = directory + f'/pretrain={pretrain}_{num}.pth'
                        )
        

        #This is so we can re-use the models we train here elsewhere...
        if pretrain==True:
            main.train_encoder()
            bt_model_path = directory+f'/bt_model_{i}'
            torch.save(main.bt_model.state_dict(), bt_model_path) #i.e. bt_model_{0,1,2 etc}
        else:
            bt_model_path=None

        metrics = main.fine_tune()
        
        metrics['bt_model_path'] =  bt_model_path #only interesting under bt pretraining...

        main_dict[i] = metrics

    return main_dict
        

In [ ]:
lst = [False,True] #pretrain or not
results={False:None,True:None} 

for pretrain in lst:
    main_dict = run_main_train(pretrain=pretrain,num_epochs=300,freeze_num_epochs=1,freeze_numfit=6,numfit=75,num=6)
    results[pretrain] = main_dict #main_dict has result of running main num=3 times.

save_dict_to_gdrive(d=results,directory=directory,filename='results')

In [26]:
results = load_dict_from_gdrive(directory=directory,filename='results')

xval = results[False][0]['xval']
yval = results[False][0]['yval']

lst = [False,True] #pretrain or not


In [ ]:
results[False][0].keys()

In [ ]:
# #How to load
# results = load_dict_from_gdrive(directory=directory,filename='results')
# path = results[False][0]['classif_model_path']
# # #init model
# _,encoder = create_model(which_model='no_pretrain',ps=8192,device=device)
# model = LM(encoder)
# model.load_state_dict(torch.load(path, map_location='cuda:0'))

In [ ]:
results[True][0]['bt_model_path']

In [ ]:
#How to load
#results = load_dict_from_gdrive(directory=directory,filename='results')
# path = results[False][0]['classif_model_path']
# # #init model
# _,encoder = create_model(which_model='no_pretrain',ps=8192,device=device)
# model = LM(encoder)
# model.load_state_dict(torch.load(path, map_location='cuda:0'))

#Print out result of each run:

In [27]:
from statistics import mean,stdev
print('random initial weights: ')
for pretrain in lst:

    _results = [results[pretrain][i]['acc'] for i in range(len(results[pretrain]))]

    print(f"\npretrain={pretrain}, results are: {_results}")
    print(f"mean is: {mean(_results)}")
    print(f"std is: {stdev(_results)}/n")


random initial weights: 

pretrain=False, results are: [0.45402297377586365, 0.44252872467041016, 0.43103447556495667, 0.43103447556495667, 0.43103447556495667, 0.4655172526836395]
mean is: 0.44252872963746387
std is: 0.014539212290958248/n

pretrain=True, results are: [0.4655172526836395, 0.568965494632721, 0.540229856967926, 0.48275861144065857, 0.5114942789077759, 0.5114942789077759]
mean is: 0.5134099622567495
std is: 0.037540060075574234/n


In [28]:
from itertools import combinations

print('Ensembling within type of initial weights')

def within_ensemble_results(results,key,param='pretrain'):

    print(f'Results for {param}={key}')

    _results = list(results[key].values())
    _results = list(combinations(_results,2)) #all pairs of results. So for num=3, will be 3
    for v in _results:

        print(f"\nAcc of first guy in ensemble is: {v[0]['acc']}")
        print(f"Acc of second guy in ensemble is: {v[1]['acc']}")
        _,acc = predict_ensemble(yval=yval,scores1=v[0]['scores'],scores2=v[1]['scores'])
        print(f'Acc of ensemble is:{acc}\n')

Ensembling within type of initial weights


In [29]:
within_ensemble_results(results,key=False)

Results for pretrain=False

Acc of first guy in ensemble is: 0.45402297377586365
Acc of second guy in ensemble is: 0.44252872467041016
Acc of ensemble is:0.4712643623352051


Acc of first guy in ensemble is: 0.45402297377586365
Acc of second guy in ensemble is: 0.43103447556495667
Acc of ensemble is:0.4597701132297516


Acc of first guy in ensemble is: 0.45402297377586365
Acc of second guy in ensemble is: 0.43103447556495667
Acc of ensemble is:0.4655172526836395


Acc of first guy in ensemble is: 0.45402297377586365
Acc of second guy in ensemble is: 0.43103447556495667
Acc of ensemble is:0.44252872467041016


Acc of first guy in ensemble is: 0.45402297377586365
Acc of second guy in ensemble is: 0.4655172526836395
Acc of ensemble is:0.48275861144065857


Acc of first guy in ensemble is: 0.44252872467041016
Acc of second guy in ensemble is: 0.43103447556495667
Acc of ensemble is:0.4712643623352051


Acc of first guy in ensemble is: 0.44252872467041016
Acc of second guy in ensemble is: 0.

In [ ]:
within_ensemble_results(results,key=True)

In [30]:
import itertools
print('Ensembling between')

def between_ensemble_results(results,key1,key2,param='pretrain'):

    #could use zip here also for uniformity

    _results = list(itertools.product(results[key1].values(),results[key2].values()))

    for v in _results:

        print(f"\nAcc of first guy in ensemble ({key1}) is: {v[0]['acc']}")
        print(f"Acc of second guy in ensemble ({key2}) is: {v[1]['acc']}")
        _,acc = predict_ensemble(yval=yval,scores1=v[0]['scores'],scores2=v[1]['scores'])
        print(f'Acc of ensemble is:{acc}\n')


 

Ensembling between


Between ensemble doesn't matter as much here: i.e. between is between pretrain=True and pretrain=False

In [31]:
between_ensemble_results(results,key1=True,key2=False)


Acc of first guy in ensemble (True) is: 0.4655172526836395
Acc of second guy in ensemble (False) is: 0.45402297377586365
Acc of ensemble is:0.4885057508945465


Acc of first guy in ensemble (True) is: 0.4655172526836395
Acc of second guy in ensemble (False) is: 0.44252872467041016
Acc of ensemble is:0.4885057508945465


Acc of first guy in ensemble (True) is: 0.4655172526836395
Acc of second guy in ensemble (False) is: 0.43103447556495667
Acc of ensemble is:0.5


Acc of first guy in ensemble (True) is: 0.4655172526836395
Acc of second guy in ensemble (False) is: 0.43103447556495667
Acc of ensemble is:0.4885057508945465


Acc of first guy in ensemble (True) is: 0.4655172526836395
Acc of second guy in ensemble (False) is: 0.43103447556495667
Acc of ensemble is:0.477011501789093


Acc of first guy in ensemble (True) is: 0.4655172526836395
Acc of second guy in ensemble (False) is: 0.4655172526836395
Acc of ensemble is:0.477011501789093


Acc of first guy in ensemble (True) is: 0.568965494